# Neural Network Autoencoder Homework

This notebook serves as the skeleton for implementing an 8‑3‑8 autoencoder from scratch using NumPy. Each section corresponds to a step in building, training, and analyzing the network. Fill in the code cells with your implementation and results.

## Overview

The goal is to implement and train a neural network with one hidden layer to reproduce the input one‑hot vectors. The network architecture consists of:

- An input layer of 8 units.
- A hidden layer with 3 units (plus a bias).
- An output layer of 8 units.

The tasks include:

1. Generating the dataset of one‑hot vectors.
2. Initializing network parameters.
3. Implementing activation functions, loss, and backpropagation.
4. Training the network using gradient descent.
5. Monitoring learning performance and plotting the loss curve.
6. Analyzing the hidden layer activations and learned weights.

Follow the sections below to complete the assignment.

In [1]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt

# Set a random seed for reproducibility (optional)
np.random.seed(42)

## Generate Dataset

Create the dataset consisting of the 8 one‑hot vectors. Each example has 7 zeros and 1 one. The output you aim for is the same as the input.

In [ ]:
# Generate 8 one‑hot vectors as input and target data
X = np.eye(8)  # shape (8, 8)
Y = X.copy()    # same as input

print('Input dataset X:')
print(X)

## Initialize Parameters

Initialize the weights and biases for the network. Use a suitable initialization strategy (e.g., Xavier/Glorot initialization) to help the network converge.

In [ ]:
# Initialize weights and biases
# W1: hidden layer weights (shape 3×8), b1: hidden layer biases (shape 3×1)
# W2: output layer weights (shape 8×3), b2: output layer biases (shape 8×1)

input_size = 8
hidden_size = 3
output_size = 8

# Xavier/Glorot initialization for sigmoid activation
limit_W1 = np.sqrt(6 / (input_size + hidden_size))
limit_W2 = np.sqrt(6 / (hidden_size + output_size))

W1 = np.random.uniform(-limit_W1, limit_W1, (hidden_size, input_size))
b1 = np.zeros((hidden_size, 1))
W2 = np.random.uniform(-limit_W2, limit_W2, (output_size, hidden_size))
b2 = np.zeros((output_size, 1))

# Print shapes to verify
print('W1 shape:', W1.shape)
print('b1 shape:', b1.shape)
print('W2 shape:', W2.shape)
print('b2 shape:', b2.shape)

## Activation and Loss Functions

Implement the sigmoid activation function and the binary cross‑entropy loss. These will be used in the forward pass and the computation of gradients.

In [ ]:
# Sigmoid activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Binary cross‑entropy loss (averaged over outputs and samples)
def bce_loss(y, y_hat):
    # Add a small epsilon to avoid log(0)
    eps = 1e-12
    y_hat = np.clip(y_hat, eps, 1 - eps)
    return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))

# Example: test activation and loss
z_example = np.array([[0], [1], [-1]])
a_example = sigmoid(z_example)
print('Sigmoid example:', a_example.flatten())

y_true = np.array([[1, 0], [0, 1]])
y_pred = np.array([[0.9, 0.1], [0.1, 0.9]])
print('BCE loss example:', bce_loss(y_true, y_pred))

## Forward and Backpropagation

Implement the forward pass to compute the network outputs given the inputs. Then implement backpropagation to compute gradients of the loss with respect to weights and biases. Use vectorized operations for efficiency.

In [ ]:
# Forward propagation
def forward_pass(X, W1, b1, W2, b2):
    # X: shape (8, N) or (N, 8) depending on representation
    # Convert X to shape (input_size, N) for consistency
    if X.shape[0] != input_size:
        X = X.T
    Z1 = W1 @ X + b1
    A1 = sigmoid(Z1)
    Z2 = W2 @ A1 + b2
    Y_hat = sigmoid(Z2)
    return Z1, A1, Z2, Y_hat

# Backpropagation
def backpropagation(X, Y, Z1, A1, Y_hat, W2):
    N = X.shape[1]  # number of samples
    
    # Compute output error
    dZ2 = (Y_hat - Y) / (output_size * N)  # BCE derivative and average
    dW2 = dZ2 @ A1.T
    db2 = np.sum(dZ2, axis=1, keepdims=True)
    
    # Compute hidden error
    dA1 = W2.T @ dZ2
    dZ1 = dA1 * A1 * (1 - A1)
    dW1 = dZ1 @ X.T
    db1 = np.sum(dZ1, axis=1, keepdims=True)
    
    return dW1, db1, dW2, db2

# Example usage: do one forward and backprop step
Z1, A1, Z2, Y_hat = forward_pass(X, W1, b1, W2, b2)
dW1, db1_grad, dW2, db2_grad = backpropagation(X.T, Y.T, Z1, A1, Y_hat, W2)
print('dW1 shape:', dW1.shape)

## Training Loop

Implement the gradient descent loop to update parameters until the network converges. Monitor the loss at each epoch and stop when a desired threshold is reached or after a fixed number of epochs.

In [ ]:
# Training parameters
learning_rate = 0.5
max_epochs = 5000
loss_history = []

# Transpose data for easier math (columns are samples)
X_train = X.T  # shape (8, 8)
Y_train = Y.T  # shape (8, 8)

for epoch in range(max_epochs):
    # Forward pass
    Z1, A1, Z2, Y_hat = forward_pass(X_train, W1, b1, W2, b2)
    
    # Compute loss
    loss = bce_loss(Y_train, Y_hat)
    loss_history.append(loss)
    
    # Backpropagation
    dW1, db1_grad, dW2, db2_grad = backpropagation(X_train, Y_train, Z1, A1, Y_hat, W2)
    
    # Update parameters
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2_grad
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1_grad
    
    # Check convergence criterion (e.g., loss threshold)
    if loss < 1e-3:
        print(f'Early stopping at epoch {epoch+1} with loss {loss:.6f}')
        break

print('Final loss:', loss_history[-1])

## Plot Loss Curve

Visualize the decrease in loss over training epochs to observe convergence behavior.

In [ ]:
plt.figure()
plt.plot(loss_history)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Binary Cross‑Entropy Loss')
plt.show()

## Evaluate and Interpret the Hidden Layer

After training, evaluate the network on all input vectors and analyze the activations of the hidden layer. Compare the raw activations and thresholded values (e.g., > 0.5) to see if the network has learned a binary encoding for each input.

In [ ]:
# Compute hidden activations and outputs
Z1, A1, Z2, Y_hat_final = forward_pass(X_train, W1, b1, W2, b2)

# Hidden activations for each input
print('Hidden activations (raw):')
print(A1)

# Threshold the activations to form a binary code
hidden_codes = (A1 > 0.5).astype(int)
print('Hidden activations (thresholded > 0.5):')
print(hidden_codes)

# Compare final outputs to targets
print('Final outputs:')
print(np.round(Y_hat_final, 3))
print('Targets:')
print(Y_train)

## Discussion and Analysis

In this section, discuss the performance of your network:

- How many epochs were required for convergence?
- How did the choice of learning rate affect convergence?
- Did different random initializations influence convergence time or final performance?
- What patterns do you observe in the hidden activations?
- Does the network learn a form of binary encoding?

Provide insights and observations based on your experiments.

## Conclusions

Summarize your findings, including the effectiveness of the autoencoder in reconstructing the input patterns, and reflect on how well the hidden layer captures the underlying structure of the data. Discuss any improvements or further experiments you would like to perform.